# Data Extraction and NLP

#### Author : Nusriya.K (nusry.k@gmail.com)

### Task : The objective of the assignment is to extract textual data article  from the given URL and perform text analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/BlackCoffer

/content/drive/MyDrive/Colab Notebooks/BlackCoffer


In [ ]:
#from google.colab import files
#uploaded=files.upload()

In [ ]:
#!unzip -uq "/content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords.zip" -d "/content/drive/MyDrive/Colab Notebooks/BlackCoffer"

In [ ]:
#!unzip -uq "/content/drive/MyDrive/Colab Notebooks/BlackCoffer/MasterDictionary.zip" -d "/content/drive/MyDrive/Colab Notebooks/BlackCoffer"

In [ ]:
#import library
import pandas as pd
import numpy as np
import string
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_excel("Input.xlsx")

In [ ]:
df.shape

(114, 2)

In [ ]:
df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [ ]:
df.tail()

,URL_ID,URL
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...
113,52768.2,https://insights.blackcoffer.com/continued-dem...


In [ ]:
df.dtypes

URL_ID    float64
URL        object
dtype: object

In [ ]:
# To check any Null Values
df.isnull().sum()

URL_ID    0
URL       0
dtype: int64

In [ ]:
# To check any duplicate rows
df.duplicated().sum()


0

### Data Extraction

In [ ]:
#extracting data from all url
import requests
from bs4 import BeautifulSoup
docs=[]#empty list
#loop through each row in the df
for index,row in df.iterrows():
  url_id=row['URL_ID']
  url = row['URL']
  #make a GET request to fetch the raw html content
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  response=requests.get(url)
  #parse the html content
  soup = BeautifulSoup(response.text, 'html.parser')
  #title from soup
  title=soup.title.text
  #get text out of the soup
  text=soup.get_text()
  #adding text to datafrmae
  docs.append(text)

print(len(docs))

114


### Data Preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
#making the stopword set from basic english and the given list of stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#adding more stopwords from text file of stopwords
import glob
import os
file_list = glob.glob(r"/content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/*.txt")

# get the file name with os.path.basename
for file_name in file_list:
    print("Full file name with directories: ", file_name)
    file_name_absolute = os.path.basename(file_name)
    print("\t\t Absolute file name: ", file_name_absolute)
    print('\n')

#extend the stop_words list
for file in file_list:
    with open(file, 'r',encoding='Latin-1') as f:
        stop_words.extend(f.read().lower().splitlines())

print("First 50 stop_words:\t",stop_words[:50])
print("\nLast 50 stop_words:\t",stop_words[-50:])

Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/StopWords_Names.txt
		 Absolute file name:  StopWords_Names.txt


Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/StopWords_Geographic.txt
		 Absolute file name:  StopWords_Geographic.txt


Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/StopWords_GenericLong.txt
		 Absolute file name:  StopWords_GenericLong.txt


Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/StopWords_DatesandNumbers.txt
		 Absolute file name:  StopWords_DatesandNumbers.txt


Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/StopWords_Currencies.txt
		 Absolute file name:  StopWords_Currencies.txt


Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/StopWords/StopWords_Auditor.txt
		 Absolute file name:  St

In [ ]:
from nltk.tokenize import word_tokenize
import re
import string

# create preprocess_text function
def preprocess_text(text):
  clean_text = re.sub(r'[^\w\s]',' ',text)
  # Tokenize the text
  text_tokens = word_tokenize(clean_text.lower())
  # Remove stop words
  filtered_tokens = [word for word in text_tokens if word not in stop_words]
  # Join the tokens back into a string(sentence)
  filtered_text= ' '.join(filtered_tokens)

  return filtered_text

# apply the function on docs
docs_1=list(map(preprocess_text,docs))
len(docs_1)


114

In [ ]:
#docs_1[113]

In [ ]:
#store positive, Negative words from the directory
pos_dict=[]
neg_dict=[]
file_list = glob.glob("/content/drive/MyDrive/Colab Notebooks/BlackCoffer/MasterDictionary/*.txt")

for file_name in file_list:
    print("Full file name with directories: ", file_name)
    file_name_absolute = os.path.basename(file_name)
    print("\t\t Absolute file name: ", file_name_absolute)

    if (file_name_absolute=='positive-words.txt'):
      with open(file_name, 'r',encoding='Latin-1') as f:
        pos_dict.extend(f.read().lower().splitlines())
    else :
        with open(file, 'r',encoding='Latin-1') as f:
         neg_dict.extend(f.read().lower().splitlines())
    print('\n')
#print("+ve words\n",pos_dict)
#print("-ve words\n",neg_dict)
print ("\n first 50 +ve words:\t",pos_dict[:50])
print ("\n first 50 -ve words:\t",neg_dict[:50])

Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/MasterDictionary/positive-words.txt
		 Absolute file name:  positive-words.txt


Full file name with directories:  /content/drive/MyDrive/Colab Notebooks/BlackCoffer/MasterDictionary/negative-words.txt
		 Absolute file name:  negative-words.txt



 first 50 +ve words:	 ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous']

 

In [ ]:
# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through each docs
for i,text in enumerate(docs_1):
  positive_words.append([word for word in text.split() if word  in pos_dict])
  negative_words.append([word for word in text.split() if word  in neg_dict])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))


In [ ]:
# Word_Count
# Average Sentence Length = the number of words / the number of sentences
# Average Word Length= Sum of the total number of characters in each  word /Total number of words
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

word_count=[]
average_sentence_length= []
average_word_length=[]
complex_word_count=[]
Percentage_Complex_words  =  []
Fog_Index = []
average_syllable_word_count =[]

stopwords = nltk.corpus.stopwords.words('english')

def measure(text):
# remove punctuations
    text = re.sub(r'[^\w\s.]',' ',text)
# split the given text file into sentences
    sentences=[]
    sentences = text.split('.')
# total number of sentences in a file
    num_sentences = len(sentences)

# total words in the file
    words=[]
    words = [word  for word in text.split() if word.lower() not in stopwords ]
    num_words = len(words)
#sum of total number of characters in each word
    length= sum(len(word) for word in words)



# complex words having syllable count is greater than 2
# Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word

# Calculations:
    word_count.append(num_words)

    avg_sent_len = num_words / num_sentences
    average_sentence_length.append(avg_sent_len)

    avg_word_length=length/num_words
    average_word_length.append(avg_word_length)

    complex_word_count.append(len(complex_words))

    avg_syllable_word_count = syllable_count / len(syllable_words)
    average_syllable_word_count.append(avg_syllable_word_count)


    Percent_Complex_words  =  len(complex_words) / num_words
    Percentage_Complex_words.append(Percent_Complex_words)

    Fog_index = 0.4 * (avg_sent_len + Percent_Complex_words)
    Fog_Index.append(Fog_index)


In [ ]:
# iterate through each file or doc
for file in docs:
  measure(file)

In [ ]:
def personal_pronoun_count(text):
  personal_pronouns = ["I", "we", "my", "ours", "us"]
  count = 0
  for pronoun in personal_pronouns:
    count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

# apply the function on docs
pp_count=list(map(personal_pronoun_count,docs))

In [ ]:
output_df = pd.read_excel('Output Data Structure.xlsx')

In [ ]:
# These are the required parameters
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            average_sentence_length,
            Percentage_Complex_words,
            Fog_Index,
            average_sentence_length,
            complex_word_count,
            word_count,
            average_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
  output_df.iloc[:,i+2] = var

# save the dataframe in excel sheet
output_df.to_excel('Output_Data.xlsx')